# Nettoyage et simplification du réseau piéton

> L’objectif de ce notebook est de nettoyer puis simplifier le réseau OSM afin d’obtenir un graphe cohérent, prêt pour les analyses de proximité sur le réseau.
> Il s’agit de la version finale et optimisée du notebook (d’autres brouillons et essais existent en local, mais celui-ci rassemble l’ensemble du travail abouti).

## Chargement des bibliothèques

On importe les outils nécessaires pour manipuler les données géographiques.

In [69]:
# Auto-enregistrement toutes les 2 minutes
%autosave 120

# Nettoie toutes les variables de la mémoire 
%reset -f

# Affichage net et clair des graphiques 
%config InlineBackend.figure_format = 'retina'

Autosaving every 120 seconds


In [70]:
import os
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString, Point, MultiPoint
from shapely.ops import split, unary_union, snap
from rtree import index as rtree_index
from tqdm import tqdm

## Validation des géométries

Avant tout traitement, je m’assure que les géométries du réseau sont valides.
Certaines peuvent être vides, corrompues ou mal définies (ex. : lignes sans coordonnées).
On les écarte pour éviter les erreurs plus tard.

In [71]:
def validate_geometry(geom):
    try:
        if geom is None or geom.is_empty or not geom.is_valid:
            return None
        return geom
    except Exception as e:
        print(f"Erreur lors de la validation de la géométrie : {e}")
        return None

Testons rapidement la fonction :

In [72]:
print(validate_geometry(LineString([(0,0),(1,1)])))   #  Ligne valide
print(validate_geometry(LineString([])))              #  Ligne vide

LINESTRING (0 0, 1 1)
None


## Extraction des lignes

On isole toutes les géométries linéaires (routes, chemins…) contenues dans la base OSM.
Les objets MultiLineString sont “dépliés” pour travailler sur des tronçons simples (LineStrings). Ceci est possible grace à la librairie `shapely`.

In [73]:
def extract_lines(roads):
    roads['geometry'] = roads['geometry'].apply(validate_geometry)
    roads = roads.dropna(subset=['geometry'])

    lines, line_ids = [], []

    for idx, row in tqdm(roads.iterrows(), total=roads.shape[0], desc="Extraction des lignes"):
        geom = row.geometry
        if geom.geom_type == 'LineString':
            lines.append(geom)
            line_ids.append(row.osm_id)
        elif geom.geom_type == 'MultiLineString':
            for sub_geom in geom.geoms:
                lines.append(sub_geom)
                line_ids.append(row.osm_id)
    return lines, line_ids

## Calcul des intersections

Cette étape est la plus délicate :
on identifie les points d’intersection entre tronçons, en tenant compte des ponts, tunnels et niveaux (layer).

Cela permet de découper plus tard le réseau sans créer de faux croisements. Les reagles de recherche d'intersections decoupages sont definis dans la section précedente.

In [74]:
def compute_intersections(lines, line_index, roads_attributes, line_ids):
    intersection_points = []

    # Valeurs spécifiques à ignorer ou à traiter à part
    tunnel_vals = ['yes', 'passage', 'covered', 'building_passage', 'cave']
    bridge_vals = ['yes', 'viaduct', 'movable', 'covered', 'service']

    def get_boundary_points(geom):
        boundary = geom.boundary
        if boundary.geom_type == 'Point':
            return [boundary]
        elif boundary.geom_type == 'MultiPoint':
            return list(boundary.geoms)
        else:
            return []

    def parse_layers(layer_val):
        if not layer_val:
            return [0.0]
        return [float(val) for val in str(layer_val).split(';') if val.strip()]

    def layers_are_compatible(layer_i, layer_j, tolerance=1):
        layers_i = parse_layers(layer_i)
        layers_j = parse_layers(layer_j)
        for li in layers_i:
            for lj in layers_j:
                if abs(li - lj) <= tolerance:
                    return True
        return False

    for i, line in tqdm(enumerate(lines), total=len(lines), desc="Calcul des intersections"):
        possible_indexes = list(line_index.intersection(line.bounds))

        for j in possible_indexes:
            if i >= j:
                continue

            id_i, id_j = line_ids[i], line_ids[j]
            attr_i, attr_j = roads_attributes.get(id_i, {}), roads_attributes.get(id_j, {})
            inter = line.intersection(lines[j])

            if inter.is_empty:
                continue

            inter_points = [inter] if inter.geom_type == 'Point' else \
                           list(inter.geoms) if inter.geom_type == 'MultiPoint' else []

            if not inter_points:
                continue

            boundary_i = get_boundary_points(line)
            boundary_j = get_boundary_points(lines[j])
            is_at_endpoint = any(pt.equals(b) for pt in inter_points for b in boundary_i + boundary_j)

            # Cas des escaliers : toujours connectés
            if attr_i.get('highway') == 'steps' or attr_j.get('highway') == 'steps':
                intersection_points.extend(inter_points)
                continue

            # Intersections en extrémité (tolérance ±1 sur les couches)
            if is_at_endpoint:
                if layers_are_compatible(attr_i.get('layer', '0'), attr_j.get('layer', '0'), tolerance=1):
                    intersection_points.extend(inter_points)
                continue

            # Autres intersections (couches strictement égales)
            if not layers_are_compatible(attr_i.get('layer', '0'), attr_j.get('layer', '0'), tolerance=0):
                continue
            if (attr_i.get('tunnel') in tunnel_vals and attr_j.get('tunnel', '') not in tunnel_vals) or \
               (attr_j.get('tunnel') in tunnel_vals and attr_i.get('tunnel', '') not in tunnel_vals):
                continue
            if (attr_i.get('bridge') in bridge_vals and attr_j.get('bridge', '') not in bridge_vals) or \
               (attr_j.get('bridge') in bridge_vals and attr_i.get('bridge', '') not in bridge_vals):
                continue

            intersection_points.extend(inter_points)

    print(f" {len(intersection_points)} intersections détectées.")
    return intersection_points

## Découpage des lignes aux intersections

Maintenant que les intersections sont identifiées,
on découpe les lignes pour obtenir des tronçons homogènes, bien connectés aux nœuds du réseau.

In [75]:
def build_point_index(pts):
    pt_index = rtree_index.Index()
    for i, pt in enumerate(tqdm(pts, desc="Indexation spatiale des points")):
        pt_index.insert(i, pt.bounds)
    return pt_index

In [76]:
def get_local_intersections(line, pts, pt_index, buffer=0.001):
    bbox = tuple([coord + delta for coord, delta in zip(line.bounds, (-buffer, -buffer, buffer, buffer))])
    candidate_ids = list(pt_index.intersection(bbox))
    selected_points = [pts[i] for i in candidate_ids if line.distance(pts[i]) < buffer]

    if selected_points:
        union_local = unary_union(selected_points)
        if union_local.geom_type == 'Point':
            union_local = MultiPoint([union_local])
        return union_local
    return None

In [77]:

def split_lines_at_intersections(lines, line_ids, pts, roads_attributes):
    tolerance = 1e-7
    final_segments, final_attrs = [], []
    pt_index = build_point_index(pts)

    for osm_id, line in tqdm(zip(line_ids, lines), total=len(lines), desc="Découpage des lignes"):
        local_intersections = get_local_intersections(line, pts, pt_index, buffer=0.001)

        if local_intersections:
            snapped_line = snap(line, local_intersections, tolerance)
            try:
                split_result = split(snapped_line, local_intersections)
            except Exception as e:
                print(f"Erreur lors du split (osm_id={osm_id}) : {e}")
                continue

            for segment in split_result.geoms:
                if segment.length > 0:
                    attr = roads_attributes.get(osm_id, {}).copy()
                    attr['osm_id'] = osm_id
                    final_segments.append(segment)
                    final_attrs.append(attr)
        else:
            attr = roads_attributes.get(osm_id, {}).copy()
            attr['osm_id'] = osm_id
            final_segments.append(line)
            final_attrs.append(attr)

    print(f"{len(final_segments)} segments obtenus après découpage.")
    return final_segments, final_attrs

## Fonction principale de simplification

Cette fonction réunit tout le processus :

 - projection en Lambert 93,

 - extraction des lignes,

 - détection d’intersections,

- découpage,

- suppression des doublons géométriques.

In [78]:
def simplify_roads(roads):
    if roads.crs.to_epsg() != 2154:
        print("Reprojection en Lambert 93 (EPSG:2154)...")
        roads = roads.to_crs(epsg=2154)

    cols_to_include = [col for col in roads.columns if col != 'geometry']
    roads_attributes = roads[cols_to_include].set_index('osm_id').to_dict('index')

    lines, line_ids = extract_lines(roads)

    # Index spatial des lignes
    line_index = rtree_index.Index()
    for pos, line in enumerate(tqdm(lines, desc="Indexation spatiale des lignes")):
        line_index.insert(pos, line.bounds)

    # Calcul et découpage
    intersection_points = compute_intersections(lines, line_index, roads_attributes, line_ids)
    final_segments, final_attrs = split_lines_at_intersections(lines, line_ids, intersection_points, roads_attributes)

    final_network_gdf = gpd.GeoDataFrame(final_attrs, geometry=final_segments, crs=roads.crs)

    print("Suppression des géométries dupliquées...")
    final_network_gdf['geom_wkb'] = final_network_gdf.geometry.apply(lambda g: g.wkb)
    final_network_gdf = final_network_gdf.drop_duplicates(subset=['geom_wkb']).drop(columns=['geom_wkb'])

    print(f"Réseau final : {len(final_network_gdf)} segments")
    return final_network_gdf

## Exemple

In [79]:
print("Chargement du réseau brut...")
roads = gpd.read_file("../../Proximity/data/processed/roads_network/osm_foot_network_Marseille_2025-10-06.gpkg")
roads.head()

Chargement du réseau brut...


,osm_id,highway,name,access,foot,surface,incline,smoothness,width,sidewalk,crossing,lit,bridge,tunnel,layer,oneway,geometry
0,2682531,service,,,,,,,,,,,,,0,no,"LINESTRING (913982.648 6247541.048, 914032.152..."
1,2699768,path,,,,,,,,,,,,,0,no,"LINESTRING (909518.351 6239066.723, 909533.383..."
2,2699769,path,,,,,,,,,,,,,0,no,"LINESTRING (909448.07 6239122.435, 909435.263 ..."
3,3617343,primary,,,,asphalt,,,,,,,,,0,no,"LINESTRING (910535.741 6250520.857, 910540.587..."
4,3617344,primary,,,,,,,,,,,,,0,no,"LINESTRING (910571.976 6250128.531, 910573.41 ..."


In [80]:
print("Simplification du réseau...")
simplified = simplify_roads(roads)

Simplification du réseau...


Calcul des intersections: 100%|██████████| 96901/96901 [00:16<00:00, 5830.82it/s] 


 159221 intersections détectées.


Découpage des lignes: 100%|██████████| 96901/96901 [00:23<00:00, 4091.34it/s]


190982 segments obtenus après découpage.
Suppression des géométries dupliquées...
Réseau final : 190945 segments


In [81]:
# Réseau traité
simplified.head()

,highway,name,access,foot,surface,incline,smoothness,width,sidewalk,crossing,lit,bridge,tunnel,layer,oneway,osm_id,geometry
0,service,,,,,,,,,,,,,0,no,2682531,"LINESTRING (913982.648 6247541.048, 914032.152..."
1,path,,,,,,,,,,,,,0,no,2699768,"LINESTRING (909518.351 6239066.723, 909533.383..."
2,path,,,,,,,,,,,,,0,no,2699769,"LINESTRING (909448.07 6239122.435, 909435.263 ..."
3,primary,,,,asphalt,,,,,,,,,0,no,3617343,"LINESTRING (910535.741 6250520.857, 910540.587..."
4,primary,,,,asphalt,,,,,,,,,0,no,3617343,"LINESTRING (910557.379 6250589.626, 910560.121..."


In [82]:
print("Enregistrement du réseau nettoyé...")
simplified.to_file("osm_foot_network_Marseille_simplified_2025-10-06.gpkg", driver="GPKG")

Enregistrement du réseau nettoyé...
